In [ ]:
import pandas as pd

In [ ]:
ofo = pd.read_csv("ofo.csv")

In [ ]:
ofo.columns = ["title", "datetime", "content"]

In [ ]:
ofo.head()

In [ ]:
ofo.shape

In [ ]:
mobike = pd.read_csv("mobike.csv")

In [ ]:
mobike.columns = ["title", "datetime", "content"]

In [ ]:
mobike.head()

In [ ]:
mobike.shape

In [ ]:
merged = pd.concat([ofo, mobike]).reset_index().drop('index', axis=1)

In [ ]:
merged.head()

In [ ]:
merged.shape

In [ ]:
merged[merged.title.duplicated()]

In [ ]:
merged = merged[~merged.title.duplicated()]

In [ ]:
merged.shape

In [ ]:
def count_ofo(mystr):
    mycount = mystr.count('OFO') +  mystr.count('ofo')
    return mycount

In [ ]:
merged["count_ofo"] = merged.content.apply(count_ofo)

In [ ]:
def count_mobike(mystr):
    mycount = mystr.count('摩拜') + mystr.count('mobike')
    return mycount

In [ ]:
merged["count_mobike"] = merged.content.apply(count_mobike)

In [ ]:
merged.head()

In [ ]:
merged[merged.count_mobike > merged.count_ofo].shape

In [ ]:
mobike = merged[merged.count_mobike > merged.count_ofo]

In [ ]:
mobike = mobike.reset_index().drop(['index','count_ofo','count_mobike'], axis=1)

In [ ]:
mobike.head()

In [ ]:
mobike.shape

In [ ]:
merged[merged.count_mobike <= merged.count_ofo].shape

In [ ]:
ofo = merged[merged.count_mobike <= merged.count_ofo]

In [ ]:
ofo = ofo.reset_index().drop(['index','count_ofo','count_mobike'], axis=1)

In [ ]:
ofo.head()

In [ ]:
ofo.shape

# Sentiment Analysis

## snownlp

In [ ]:
from snownlp import SnowNLP
def get_sentiment(text):
    return SnowNLP(text).sentiments

In [ ]:
ofo["sentiment"] = ofo.content.apply(get_sentiment)
mobike["sentiment"] = mobike.content.apply(get_sentiment)

In [ ]:
ofo[ofo.sentiment > 0.5].shape

In [ ]:
ofo[ofo.sentiment <= 0.5].shape

In [ ]:
mobike[mobike.sentiment > 0.5].shape

In [ ]:
mobike[mobike.sentiment <= 0.5].shape

## bosonnlp

In [ ]:
# from bosonnlp import BosonNLP

# import json

# with open("boson_api_key.json") as f:
#     secret = json.load(f)

# nlp = BosonNLP(secret["boson_api"])

# def get_sentiment_bosonnlp(text):
#     return nlp.sentiment(text, model="news")[0][0]

# ofo["sentiment"] = ofo.content.apply(get_sentiment_bosonnlp)

# mobike["sentiment"] = mobike.content.apply(get_sentiment_bosonnlp)

In [ ]:
import pickle

In [ ]:
# with open('sentiment_bosonnlp.pickle', 'wb') as f:
#     pickle.dump([ofo, mobike], f)

In [ ]:
with open('sentiment_bosonnlp.pickle', 'rb') as f:
    [ofo, mobike] = pickle.load(f)

In [ ]:
ofo[ofo.sentiment > 0.5].shape

In [ ]:
ofo[ofo.sentiment <= 0.5].shape

In [ ]:
mobike[mobike.sentiment > 0.5].shape

In [ ]:
mobike[mobike.sentiment <= 0.5].shape

# Visualization

In [ ]:
%matplotlib inline

In [ ]:
ofo.set_index('datetime', inplace=True)

In [ ]:
mobike.set_index('datetime', inplace=True)

In [ ]:
(ofo[['sentiment']] - 0.5).plot(kind='bar', figsize=(24, 6))

In [ ]:
(mobike[['sentiment']] - 0.5).plot(kind='bar', figsize=(24, 6))

In [ ]:
import seaborn as sns

In [ ]:
ofo["brand"] = "ofo"

In [ ]:
mobike["brand"] = "mobike"

In [ ]:
ofo.head()

In [ ]:
df = pd.concat([ofo, mobike])

In [ ]:
sns.boxplot(x='brand', y='sentiment', data=df)

In [ ]:
ofo_positive = ofo[ofo.sentiment > 0.5]
ofo_negative = ofo[ofo.sentiment <= 0.5]
mobike_positive = mobike[mobike.sentiment > 0.5]
mobike_negative = mobike[mobike.sentiment <= 0.5]

In [ ]:
ofo_positive.shape

In [ ]:
ofo_negative.shape

In [ ]:
mobike_positive.shape

In [ ]:
mobike_negative.shape

# LDA

In [ ]:
import pyLDAvis 
import pyLDAvis.sklearn 

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
from helper import *

In [ ]:
stopwords = get_custom_stopwords("stopwordsHIT.txt") # HIT停用词词典
max_df = 0.7 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 2 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
n_features = 1000 # 最大提取特征数量
n_top_words = 20 # 显示主题下关键词的时候，显示多少个
col_content = "content" # 说明其中的文本信息所在列名称

In [ ]:
def lda_on_chinese_articles(df, n_topics):
    return lda_on_chinese_articles_with_param(df, n_topics, 
                            col_content = col_content, 
                            stopwords = stopwords, 
                            n_features = n_features, 
                            max_df = max_df, 
                            min_df = min_df,
                            n_top_words = n_top_words)

In [ ]:
df = ofo_positive
n_topics = 4
lda, tf, vect = lda_on_chinese_articles(df = df, n_topics = n_topics)
pyLDAvis.sklearn.prepare(lda, tf, vect)

In [ ]:
df = ofo_negative
n_topics = 3
lda, tf, vect = lda_on_chinese_articles(df = df, n_topics = n_topics)
pyLDAvis.sklearn.prepare(lda, tf, vect)

In [ ]:
df = mobike_positive
n_topics = 4
lda, tf, vect = lda_on_chinese_articles(df = df, n_topics = n_topics)
pyLDAvis.sklearn.prepare(lda, tf, vect)

In [ ]:
df = mobike_negative
n_topics = 3
lda, tf, vect = lda_on_chinese_articles(df = df, n_topics = n_topics)
pyLDAvis.sklearn.prepare(lda, tf, vect)